In [1]:
## Load necessary packages
import os
import pandas as pd
import glob
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

## Define the version number
version_number = "07_08_2025"
deployment_date = "2025-07-08"

In [ ]:
## 2. Locally use deepseek LLM model, test case
## example case

import ollama
response = ollama.chat(
    model="deepseek-r1:14b",
    messages=[
        {"role": "user", "content": "Explain Newton's second law of motion"},
    ],
)
print(response["message"]["content"])

In [2]:
## Notice!! Please change the file path of following codes into your own
raw_files_path = '/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/'

In [4]:
## Check all node files being read
## Read all BigGIM node csv file in group 1

for f in os.listdir(raw_files_path):
    if f.endswith('.tsv'):
        print(f)

PharmGKB_parsed_node_05_29_2025.tsv
primeKG_parsed_node_07_08_2025.tsv
primeKG_parsed_edge_07_08_2025.tsv
PharmGKB_parsed_edge_05_29_2025.tsv
PharmGKB_parsed_edge_07_08_2025.tsv
primeKG_parsed_edge_05_29_2025.tsv
primeKG_parsed_node_05_29_2025.tsv
PharmGKB_parsed_node_07_08_2025.tsv
Pharmagenomics_KG_parsed_node_07_08_2025.tsv
Pharmagenomics_KG_parsed_edge_07_08_2025.tsv
primeKG_parsed_edge_05_26_2025.tsv
primeKG_parsed_node_05_19_2025.tsv
primeKG_parsed_edge_05_19_2025.tsv
primeKG_parsed_node_05_26_2025.tsv
Pharmagenomics_KG_parsed_edge_05_29_2025.tsv
Pharmagenomics_KG_parsed_node_05_29_2025.tsv
FDA_Pharmacogenomic_biomarkers_parsed_edge_07_14_2025.tsv
FDA_Pharmacogenomic_biomarkers_parsed_node_07_14_2025.tsv


In [7]:
## Read each individual csv files
source_df = pd.read_csv(raw_files_path + f'Pharmagenomics_KG_parsed_edge_{version_number}.tsv', sep='\t')

source_df.head(5)

/var/folders/ml/cjwyk2ps62361rr96_3b_n080000gp/T/ipykernel_36867/1161309823.py:2: DtypeWarning: Columns (11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  source_df = pd.read_csv(raw_files_path + f'Pharmagenomics_KG_parsed_edge_{version_number}.tsv', sep='\t')


,subject,object,subject_category,object_category,predicate,knowledge_source,knowledge_level,agent_type,deploy_date,context_qualifier,id,predicate_source_name,anatomical_context_qualifier,publications
0,NCBIGene:9796,NCBIGene:56992,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-07-08,NaN,b517ee25-7533-5773-9d01-55c489e65ce3,NaN,NaN,NaN
1,NCBIGene:7918,NCBIGene:9240,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-07-08,NaN,982e1505-0a46-524a-ae94-a01d3ad46662,NaN,NaN,NaN
2,NCBIGene:8233,NCBIGene:23548,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-07-08,NaN,88373117-1e09-5fbc-92c7-44196778f5db,NaN,NaN,NaN
3,NCBIGene:4899,NCBIGene:11253,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-07-08,NaN,bd1b1329-75a1-5be0-b7ad-8965cef9a0c2,NaN,NaN,NaN
4,NCBIGene:5297,NCBIGene:8601,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,PrimeKG,knowledge_assertion,automated_agent,2025-07-08,NaN,17fd6751-6b8d-5556-b51b-3f6b2113e7b1,NaN,NaN,NaN


In [ ]:
import requests

## define query function

def query_ollama(prompt, model='llama3'):
    response = requests.post(
        'http://localhost:11434/api/generate',
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()['response'].strip()

def label_row(row):
    combined_text = f"{row['subject']} {row['predicate']} {row['object']}"
    prompt = f"""Analyze the overall likelihood of the following combined text as valid or unlikely:
                \"\"\"{combined_text}\"\"\"
            Sentiment:"""
    return query_ollama(prompt)

In [ ]:

## apply to each row of the dataframe
source_df['label'] = source_df.apply(label_row, axis=1)
print(source_df)